In [12]:
import tqdm
import torch

import numpy as np
from torchvision.transforms import Compose, ToTensor
from torchvision.transforms import RandomHorizontalFlip, RandomCrop
from torchvision.transforms import Normalize
from torch.utils.data.dataloader import DataLoader
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets

from torch.optim.adam import Adam
from modules.pytorchtools import EarlyStopping # 위 링크의 깃허브 파일에서 임포트

In [2]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(BasicBlock, self).__init__()

        # 1.합성곱층 정의
        self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
        self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1)

        self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        # 2.배치 정규화층 정의
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        # 3.스킵 커넥션을 위해 초기 입력을 저장
        x_ = x

        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)

        # 4.합성곱의 결과와 입력의 채널 수를 맞춤
        x_ = self.downsample(x_)

        # 5.합성곱층의 결과와 저장해놨던 입력값을 더해줌
        x += x_
        x = self.relu(x)

        return x

In [32]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()

        #Output Size = (W - F + 2P) / S + 1
        #W: input_volume_size
        #F: kernel_size
        #P: padding_size
        #S: strides
        #nn.Conv2d(in_channels=3, out_channels=32, kernel=3, padding=1)
        #output_size = (32 - 3 + 2*1) / 1 + 1 = 32

        # ❶ 기본 블록
        # 기본 입력이 RGB인 3채널
        self.b1 = BasicBlock(in_channels=3, out_channels=64)
        self.b2 = BasicBlock(in_channels=64, out_channels=128)
        self.b3 = BasicBlock(in_channels=128, out_channels=256)


        # ❷ 풀링을 최댓값이 아닌 평균값으로
        # maxpooling=2
        #input_filter_size/2 = output_filter_size
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # ❸ 분류기  MLP
        # 4 * 4* 256
        # out_feature * kerner
        # (batchsize/2) * imgsize
        # 4 * 224 * 224
        #self.fc1 = nn.Linear(in_features= 4 * 128 * 128, out_features=2048)
        self.fc1 = nn.Linear(in_features=4 * 64 * 64, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=num_classes)

        self.relu = nn.ReLU()
    def forward(self, x):
        
        # 1. 기본 블록과 풀링층을 통과
        x = self.b1(x)
        x = self.pool(x)
        x = self.b2(x)
        x = self.pool(x)
        x = self.b3(x)
        x = self.pool(x)


        # ❷ 분류기의 입력으로 사용하기 위해 flatten
        x = torch.flatten(x, start_dim=1)

        # ❸ 분류기로 예측값 출력
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x

In [22]:
# 1024x1024 resize train / meanR=0.43303847, meanG=0.4034577, meanB=0.39415097  / stdR=0.18344551, stdG=0.17549995, stdB=0.1647388   
# 128x128 resize train / meanR=0.43305725, meanG=0.40347522, meanB=0.3941705  / stdR=0.17281055, stdG=0.16584247, stdB=0.15571058 
# 244 평균 0.27763095, 0.22682726, 0.2020654
# 표준편차 0.23883308, 0.21106692, 0.20242342

import os
import torchvision.transforms as transforms

def create_datasets(data_dir, batch_size):
    
    train_transform = transforms.Compose([
        #transforms.RandomHorizontalFlip(),  # 좌우반전 
        #transforms.RandomVerticalFlip(),  # 상하반전 
        #transforms.Resize(32),  # 알맞게 변경하세요 
        transforms.Resize(64),  # 알맞게 변경하세요 
        transforms.CenterCrop()
        transforms.ToTensor(),  # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
        transforms.Normalize([0.2749446, 0.22524835, 0.20173368], [0.2369616, 0.21033151, 0.20125428])  #  정규화(normalization)
    ])
    
    # test_transform = transforms.Compose([   # 나중에 test 데이터 불러올 때 참고하세요. 
    #     transforms.Resize(224),
    #     transforms.ToTensor(), # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화 
    #     transforms.Normalize([0.2749446, 0.22524835, 0.20173368], [0.2369616, 0.21033151, 0.20125428])  # 테스트 데이터로 계산을 진행해서 따로 지정해주어도 좋습니다
    # ])

    # choose the training and test datasets
    train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transform)

    # trainning set 중 validation 데이터로 사용할 비율
    valid_size = 0.2

    # validation으로 사용할 trainning indices를 얻는다.
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # trainning, validation batch를 얻기 위한 sampler정의
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = DataLoader(train_data,
                               batch_size=batch_size,
                               sampler=train_sampler,
                               num_workers=4)

    # load validation data in batches
    valid_loader = DataLoader(train_data,
                               batch_size=batch_size,
                               sampler=valid_sampler,
                               num_workers=4)

    return train_data, train_loader, valid_loader

In [23]:
root_dir = '/home/jupyter/workspace/eyes'
bs = 4
train_data, train_loader, valid_loader = create_datasets(root_dir, bs)

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNet(num_classes=len(train_data.classes))
model.to(device)

ResNet(
  (b1): BasicBlock(
    (c1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasicBlock(
    (c1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasicBlock(
    (c1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [34]:
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)

# early_stopping 객체 선언(3번의 epoch 연속으로 loss 미개선 시에 조기 종료 예시)
early_stopping = EarlyStopping(patience = 10, verbose = True)

for epoch in range(50):
    ### 각 epoch의 train 부분 ###
    model.train()
    val_loss = 0
    iterator = tqdm.tqdm(train_loader)
    for data, label in iterator:
        # 최적화를 위해 기울기를 초기화
        optim.zero_grad()

        # 모델의 예측값
        preds = model(data.to(device))
        
        # 손실 계산 및 역전파
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
        
        val_loss += loss.item()

        iterator.set_description(f"train epoch:{epoch+1} loss:{loss.item()}")
        
    ### 각 epoch train 이후 evaluation 진행 ###
    model.eval()
    
    ### early stopping 여부를 체크하는 부분 ###
    early_stopping(val_loss, model) # 현재 과적합 상황 추적
    
    if early_stopping.early_stop: # 조건 만족 시 조기 종료
        break

torch.save(model.state_dict(), "ResNet.pth")

train epoch:1 loss:2.072284460067749: 100% 332/332 [00:08<00:00, 39.29it/s] 


Validation loss decreased (inf --> 681.556140).  Saving model ...


train epoch:2 loss:2.5176048278808594: 100% 332/332 [00:08<00:00, 40.90it/s]


Validation loss decreased (681.556140 --> 596.854130).  Saving model ...


train epoch:3 loss:1.233520269393921: 100% 332/332 [00:08<00:00, 40.05it/s] 


...

EarlyStopping counter: 4 out of 10


train epoch:49 loss:0.0004936078912578523: 100% 332/332 [00:07<00:00, 42.46it/s] 


EarlyStopping counter: 5 out of 10


train epoch:50 loss:3.8890138966962695e-05: 100% 332/332 [00:07<00:00, 42.21it/s]


EarlyStopping counter: 6 out of 10


In [42]:
model.load_state_dict(torch.load("checkpoint.pt", map_location=device))

num_corr = 0
correct = 0

with torch.no_grad():
    for data, label in valid_loader:
        output = model(data.to(device))
        #torch.Size([3, 64, 64])
        #print(data.to(device)[0].shape)
        #print(label, output.data.max(1)[1])
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        #correct += preds.eq(label.to(device).view_as(preds)).sum().item()  # target, pred 일치하는 값의 개수
        num_corr += corr

    print(f"Accuracy:{num_corr/(len(valid_loader)*bs)}")

Accuracy:0.37349397590361444
